In [1]:
import pandas as pd
import numpy as np

In [13]:
seoul_cluster_data = pd.read_csv('./cluster_result/seoul_cluster.csv',encoding='cp949')
busan_data = pd.read_csv('./data/busan_data80.csv',encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: './cluster_result/seoul_cluster.csv'

In [ ]:
#문자 타입 숫자로 변환
busan_data = pd.get_dummies(busan_data, columns = ['강']) 

#문자 타입 숫자로 변환(하천 존재할 시 1, 존재하지 않을 시 0)
busan_data['하천'] = busan_data['하천'].fillna(0).map(lambda x: 1 if x != 0 else x)
busan_data = busan_data.fillna(0)

In [14]:
busan_data

NameError: name 'busan_data' is not defined

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

# 행정동명 열 추출
busan_hood_names = busan_data['행정동명']
busan_data = busan_data.drop(['행정동명'], axis=1) 
busan_data = busan_data.rename(columns={'강_낙동강': '강'})
busan_data = busan_data.rename(columns={'지하철승하차합계': '지하철_총승객수'})
busan_data = busan_data[['하천', '자동차등록대수'] + [col for col in busan_data.columns if col not in ['자동차등록대수', '하천']]]
busan_data = busan_data[['시설물개수', '인구밀도'] + [col for col in busan_data.columns if col not in ['인구밀도', '시설물개수']]]

# 서울 클러스터 데이터 프레임 (서울 클러스터 열 포함)
seoul_cluster_names = seoul_cluster_data['cluster']
seoul_cluster_data = seoul_cluster_data.drop(['cluster'], axis=1) 
#seoul_cluster_data = seoul_cluster_data.drop(['cluster', '행정동명','버스_총승객수'], axis=1) # 서울 클러스터와 행정동명 열 제외
#seoul_cluster_data = seoul_cluster_data.rename(columns={'강_한강': '강'})
#seoul_cluster_data = seoul_cluster_data.rename(columns={'인구밀도 (명/㎢)': '인구밀도'})
#seoul_cluster_data = seoul_cluster_data.rename(columns={'자전거도로(km)': '자전거도로'})


In [56]:
busan_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강
0,12.0,31691.011240,0,8453.769231,1.780000,1093253,5.0,0.0,1.0,0
1,11.0,6898.360656,0,8453.769231,1.780000,297383,2.0,1.0,0.0,0
2,0.0,18322.033900,0,8453.769231,1.780000,250423,4.0,0.0,0.0,0
3,13.0,32082.692310,0,8453.769231,1.780000,2260041,1.0,0.0,0.0,0
4,3.0,13609.322030,0,8453.769231,1.780000,206875,3.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
75,20.0,21574.305560,0,10137.444440,3.586667,791741,8.0,0.0,0.0,0
76,22.0,6680.501931,1,10137.444440,3.586667,10196,8.0,0.0,0.0,0
77,18.0,1930.473373,0,10137.444440,3.586667,279935,3.0,0.0,1.0,0
78,4.0,4811.764706,0,10137.444440,3.586667,137265,4.0,0.0,0.0,0


In [57]:
seoul_cluster_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강
0,9.333333,17139.33333,0.266667,7786.995966,3.207205,8.732595e+05,3.400000,0.266667,0.133333,1.000
1,18.750000,12390.87500,0.125000,6666.492361,2.599038,6.918174e+06,2.875000,1.375000,0.000000,0.375
2,14.611111,15556.72222,0.000000,8845.522968,3.513441,8.976867e+05,7.277778,0.222222,0.000000,0.000
3,10.931034,30007.55172,0.000000,7118.850769,2.676257,1.370194e+06,3.241379,0.344828,0.000000,0.000
4,11.950000,23326.30000,1.000000,7341.498996,2.613947,6.301741e+05,4.650000,0.650000,0.150000,0.000
5,15.666667,14609.77778,0.000000,6835.457310,2.113959,1.365974e+06,4.000000,0.555556,1.222222,0.000


In [58]:
# 코사인 유사도 계산 (부산 데이터와 서울 클러스터 데이터 간)
similarity_matrix = cosine_similarity(busan_data, seoul_cluster_data)

# 각 부산 행정동을 가장 유사한 서울 클러스터에 매핑
busan_mapping = similarity_matrix.argmax(axis=1)

# 매핑 결과를 부산 데이터프레임에 추가
busan_data['서울 클러스터'] = busan_mapping

# 행정동명 열 다시 추가
busan_data['행정동명'] = busan_hood_names

busan_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강,서울 클러스터,행정동명
0,12.0,31691.011240,0,8453.769231,1.780000,1093253,5.0,0.0,1.0,0,3,거제1동
1,11.0,6898.360656,0,8453.769231,1.780000,297383,2.0,1.0,0.0,0,2,거제2동
2,0.0,18322.033900,0,8453.769231,1.780000,250423,4.0,0.0,0.0,0,4,연산3동
3,13.0,32082.692310,0,8453.769231,1.780000,2260041,1.0,0.0,0.0,0,5,연산5동
4,3.0,13609.322030,0,8453.769231,1.780000,206875,3.0,0.0,0.0,0,4,연산6동
...,...,...,...,...,...,...,...,...,...,...,...,...
75,20.0,21574.305560,0,10137.444440,3.586667,791741,8.0,0.0,0.0,0,0,좌2동
76,22.0,6680.501931,1,10137.444440,3.586667,10196,8.0,0.0,0.0,0,4,반여1동
77,18.0,1930.473373,0,10137.444440,3.586667,279935,3.0,0.0,1.0,0,2,반송1동
78,4.0,4811.764706,0,10137.444440,3.586667,137265,4.0,0.0,0.0,0,4,반송2동


In [46]:
busan_data.to_csv('./cluster_result/busan_cluster80.csv',index=False)

In [59]:
# 특정 부산 행정동의 인덱스 (예: 0번째 행정동)
busan_index = 0

# 해당 부산 행정동과 서울 클러스터 간의 유사도
similarities = similarity_matrix[busan_index]

# 유사도 확인
print(similarities)

[0.99995554 0.99960754 0.99992987 0.9999717  0.9999602  0.99982913]


In [61]:
similarity_df = pd.DataFrame(similarity_matrix, columns=[f"서울클러스터{i}" for i in range(6)])

# 부산 행정동명 열 추가
similarity_df['부산행정동명'] = busan_data['행정동명']

# 데이터프레임 출력
print(similarity_df)

     서울클러스터0   서울클러스터1   서울클러스터2   서울클러스터3   서울클러스터4   서울클러스터5 부산행정동명
0   0.999956  0.999608  0.999930  0.999972  0.999960  0.999829   거제1동
1   0.999804  0.999394  0.999811  0.999730  0.999764  0.999648   거제2동
2   0.998269  0.996930  0.998166  0.998289  0.999109  0.997648   연산3동
3   0.999972  0.999919  0.999976  0.999969  0.999709  0.999993   연산5동
4   0.998433  0.997169  0.998354  0.998410  0.999165  0.997850   연산6동
..       ...       ...       ...       ...       ...       ...    ...
75  0.999963  0.999606  0.999947  0.999957  0.999952  0.999833    좌2동
76  0.656929  0.644480  0.656583  0.655494  0.665656  0.650771   반여1동
77  0.999547  0.999366  0.999599  0.999407  0.999246  0.999506   반송1동
78  0.997785  0.996807  0.997808  0.997570  0.998075  0.997348   반송2동
79  0.999775  0.999850  0.999825  0.999688  0.999316  0.999861   대저1동

[80 rows x 7 columns]


In [62]:
similarity_df.to_csv('./cluster_result/busan_cluster80_similarity.csv',index=False)

## standard 민코프스키 

In [6]:
seoul_st_cluster_data = pd.read_csv('./cluster_result/seoul_standard_cluster_5avg.csv',encoding='cp949')
busan_st_data = pd.read_csv('./data/busan_standard_data80.csv',encoding='cp949')

In [7]:
busan_dong = busan_st_data['행정동명']
busan_st_data = busan_st_data.drop(['행정동명'], axis=1) 

In [8]:
busan_st_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강
0,0.242790,2.321545,-0.258199,0.443262,-0.196669,0.879552,0.842160,-0.264363,2.645751,-0.333333
1,0.131800,-0.663052,-0.258199,0.443262,-0.196669,-0.476009,-0.487566,0.910584,-0.377964,-0.333333
2,-1.089086,0.712156,-0.258199,0.443262,-0.196669,-0.555994,0.398918,-0.264363,-0.377964,-0.333333
3,0.353780,2.368696,-0.258199,0.443262,-0.196669,2.866879,-0.930809,-0.264363,-0.377964,-0.333333
4,-0.756117,0.144829,-0.258199,0.443262,-0.196669,-0.630167,-0.044324,-0.264363,-0.377964,-0.333333
...,...,...,...,...,...,...,...,...,...,...
75,1.130707,1.103672,-0.258199,1.201830,0.718598,0.366004,2.171887,-0.264363,-0.377964,-0.333333
76,1.352687,-0.689279,3.872983,1.201830,0.718598,-0.965159,2.171887,-0.264363,-0.377964,-0.333333
77,0.908728,-1.261098,-0.258199,1.201830,0.718598,-0.505728,-0.044324,-0.264363,2.645751,-0.333333
78,-0.645127,-0.914242,-0.258199,1.201830,0.718598,-0.748730,0.398918,-0.264363,-0.377964,-0.333333


In [9]:
# 거리 계산 함수
def minkowski_distance(x, y, p=2):
    return np.power(np.sum(np.abs(x - y) ** p), 1/p)

# 부산 데이터와 서울 클러스터 데이터 간의 거리 계산
busan_distances = []
for i in range(len(busan_st_data)):
    distances = []
    for j in range(len(seoul_st_cluster_data)):
        distance = minkowski_distance(busan_st_data.iloc[i], seoul_st_cluster_data.iloc[j], p=2)
        distances.append(distance)
    busan_distances.append(distances)

# 거리를 데이터프레임으로 변환
distance_df = pd.DataFrame(busan_distances, columns=[0, 1, 2, 3, 4, 5])
distance_df['행정동명'] = busan_dong

# 각 행정동에 가장 가까운 클러스터 결정
busan_st_data['서울클러스터'] = distance_df.iloc[:, :-1].idxmin(axis=1)
busan_st_data['행정동명'] = busan_dong

for cluster in [0, 1, 2, 3, 4, 5]:
    busan_st_data[cluster] = distance_df[cluster]


# 결과 확인
print(busan_st_data)



       시설물개수      인구밀도        하천   자동차등록대수     자전거도로  지하철_총승객수      학교개수  \
0   0.242790  2.321545 -0.258199  0.443262 -0.196669  0.879552  0.842160   
1   0.131800 -0.663052 -0.258199  0.443262 -0.196669 -0.476009 -0.487566   
2  -1.089086  0.712156 -0.258199  0.443262 -0.196669 -0.555994  0.398918   
3   0.353780  2.368696 -0.258199  0.443262 -0.196669  2.866879 -0.930809   
4  -0.756117  0.144829 -0.258199  0.443262 -0.196669 -0.630167 -0.044324   
..       ...       ...       ...       ...       ...       ...       ...   
75  1.130707  1.103672 -0.258199  1.201830  0.718598  0.366004  2.171887   
76  1.352687 -0.689279  3.872983  1.201830  0.718598 -0.965159  2.171887   
77  0.908728 -1.261098 -0.258199  1.201830  0.718598 -0.505728 -0.044324   
78 -0.645127 -0.914242 -0.258199  1.201830  0.718598 -0.748730  0.398918   
79 -0.423148 -1.449013  3.872983  1.621311  5.589399  0.049120  0.398918   

       영화관개수     대학교개수         강  서울클러스터  행정동명         0         1         2  \
0  -0.2

In [10]:
busan_st_data.to_csv('./cluster_result/busan80_standard_euclidean.csv', index = False, encoding = 'cp949')

## minmax 민코프스키

In [2]:
seoul_mm_cluster_data = pd.read_csv('./cluster_result/seoul_minmax_newcluster_5avg.csv',encoding='cp949')
busan_mm_data = pd.read_csv('./data/busan_minmax_newdata.csv',encoding='cp949')

In [3]:
seoul_mm_cluster_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강,cluster
0,0.224700,0.473417,0.245283,0.379222,0.287605,0.093244,0.210877,0.089623,0.030189,0.00,0
1,0.636364,0.129922,0.250000,0.418950,0.392972,0.741752,0.117647,0.562500,0.250000,0.25,1
2,0.539627,0.411420,0.269231,0.711939,0.605819,0.079862,0.398190,0.153846,0.046154,0.00,2
3,0.247772,0.293429,0.235294,0.455774,0.443428,0.108372,0.193772,0.102941,0.023529,1.00,3


In [12]:
busan_dong = busan_mm_data['행정동명']
busan_mm_data = busan_mm_data.drop(['행정동명'], axis=1) 

In [13]:
# 거리 계산 함수
def minkowski_distance(x, y, p=2):
    return np.power(np.sum(np.abs(x - y) ** p), 1/p)

# 부산 데이터와 서울 클러스터 데이터 간의 거리 계산
busan_distances = []
for i in range(len(busan_mm_data)):
    distances = []
    for j in range(len(seoul_mm_cluster_data)):
        distance = minkowski_distance(busan_mm_data.iloc[i], seoul_mm_cluster_data.iloc[j], p=2)
        distances.append(distance)
    busan_distances.append(distances)

# 거리를 데이터프레임으로 변환
distance_df = pd.DataFrame(busan_distances, columns=[0, 1, 2, 3])
distance_df['행정동명'] = busan_dong

# 각 행정동에 가장 가까운 클러스터 결정
busan_mm_data['서울클러스터'] = distance_df.iloc[:, :-1].idxmin(axis=1)
busan_mm_data['행정동명'] = busan_dong

for cluster in [0, 1, 2, 3]:
    busan_mm_data[cluster] = distance_df[cluster]


# 결과 확인
print(busan_mm_data)

        시설물개수      인구밀도   하천   자동차등록대수     자전거도로  지하철_총승객수      학교개수  \
0    0.181818  0.758310  0.0  0.358600  0.065328  0.261994  0.357143   
1    0.166667  0.162808  0.0  0.154869  0.028928  0.071267  0.142857   
2    0.015152  0.368018  0.0  0.114989  0.021802  0.000000  0.000000   
3    0.045455  0.265502  0.0  0.118375  0.022407  0.000000  0.285714   
4    0.106061  0.458087  0.0  0.203105  0.037546  0.000000  0.285714   
..        ...       ...  ...       ...       ...       ...       ...   
201  0.469697  0.100414  0.0  0.978446  1.000000  0.000000  0.928571   
202  0.212121  0.218019  0.0  0.539537  0.555799  0.000000  0.214286   
203  0.030303  0.000076  0.0  0.029881  0.039997  0.000000  0.214286   
204  0.727273  0.012005  0.0  0.634317  0.651722  0.000000  0.357143   
205  0.030303  0.000992  0.0  0.067042  0.077606  0.000000  0.214286   

        영화관개수  대학교개수    강  서울클러스터  행정동명         0         1         2  \
0    0.000000    0.5  0.0       0  거제1동  0.686749  1.226339  1

In [14]:
busan_mm_data.to_csv('./cluster_result/busan_뉴_minmax_euclidean.csv', index = False, encoding = 'cp949')

In [4]:
seoul_st_cluster_data = pd.read_csv('./cluster_result/busan_뉴_minmax_euclidean.csv',encoding='cp949')

In [5]:
seoul_st_cluster_data

,시설물개수,인구밀도,하천,자동차등록대수,자전거도로,지하철_총승객수,학교개수,영화관개수,대학교개수,강,서울클러스터,행정동명,0,1,2,3
0,0.181818,0.758310,0.0,0.358600,0.065328,0.261994,0.357143,0.000000,0.5,0.0,0,거제1동,0.686749,1.226339,1.001162,1.310015
1,0.166667,0.162808,0.0,0.154869,0.028928,0.071267,0.142857,0.142857,0.0,0.0,0,거제2동,0.534895,1.112542,0.991990,1.160807
2,0.015152,0.368018,0.0,0.114989,0.021802,0.000000,0.000000,0.000000,0.0,0.0,0,거제3동,0.563702,1.319355,1.112475,1.212181
3,0.045455,0.265502,0.0,0.118375,0.022407,0.000000,0.285714,0.000000,0.0,0.0,0,거제4동,0.545096,1.295209,1.037063,1.191436
4,0.106061,0.458087,0.0,0.203105,0.037546,0.000000,0.285714,0.000000,0.0,0.0,0,연산1동,0.437332,1.282255,0.943036,1.167256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,0.469697,0.100414,0.0,0.978446,1.000000,0.000000,0.928571,0.285714,0.0,1.0,3,명지1동,1.639829,1.637704,1.306792,1.144755
202,0.212121,0.218019,0.0,0.539537,0.555799,0.000000,0.214286,0.000000,0.0,1.0,3,명지2동,1.113925,1.338745,1.146896,0.324470
203,0.030303,0.000076,0.0,0.029881,0.039997,0.000000,0.214286,0.000000,0.0,1.0,3,가락동,1.234100,1.491166,1.533772,0.745767
204,0.727273,0.012005,0.0,0.634317,0.651722,0.000000,0.357143,0.000000,0.0,1.0,3,녹산동,1.327528,1.321728,1.144213,0.699524
